In [1]:
src_path="Tables/dbo/accounts"
df_source = spark.read.format("delta").load(src_path)
df_source.createOrReplaceTempView("source_view")

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 75, Finished, Available, Finished)

In [7]:
create_table_query = """
CREATE TABLE IF NOT EXISTS Account_SCD (
    account_id int,
    customer_id int,
    account_type string,
    balance float,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Tables/Gold_layer/Account_SCD'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 76, Finished, Available, Finished)

DataFrame[]

In [6]:
from delta.tables import DeltaTable

target_path = "Tables/Gold_layer/Account_SCD"  
delta_target = DeltaTable.forPath(spark, target_path)

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 77, Finished, Available, Finished)

In [8]:
from pyspark.sql.functions import *
df_src1= df_source.withColumn("hash_key",crc32(concat(*df_source.columns)))
display(df_src1)

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 78, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7089d3a7-e49d-4105-be27-6025f74ac416)

In [9]:
df_src1=df_src1.alias("src").join(delta_target.toDF().alias("tgt"),((col("src.account_id")==col("tgt.account_id"))&(col("src.hash_key")==col("tgt.hash_key"))),"anti").select(col("src.*"))
df_src1.show()

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 79, Finished, Available, Finished)

+----------+-----------+------------+-------+--------+
|account_id|customer_id|account_type|balance|hash_key|
+----------+-----------+------------+-------+--------+
+----------+-----------+------------+-------+--------+



In [10]:
from pyspark.sql.functions import col

delta_target.alias("tgt").merge(df_src1.alias("src"),"tgt.account_id = src.account_id")\
    .whenMatchedUpdate(set={"tgt.account_id":"src.account_id","tgt.customer_id":"src.customer_id","tgt.account_type":"src.account_type","tgt.balance":"src.balance","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.account_id":"src.account_id","tgt.customer_id":"src.customer_id","tgt.account_type":"src.account_type","tgt.balance":"src.balance","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
display(spark.read.format("delta").option("header","true").load(target_path))

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 80, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 46be35d8-ae94-4620-91e0-2424917adfe5)

In [ ]:
from pyspark.sql.functions import *
df_source = spark.read.format("delta").load("Tables/dbo/customers")
df_source.createOrReplaceTempView("source_view")
src_path="Tables/dbo/customers"

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 81, Finished, Available, Finished)

In [40]:
create_table_query = """
CREATE TABLE IF NOT EXISTS Customer_SCD1 (
    customer_id int,
    first_name string,
    last_name string,
    address string,
    city string,
    state string,
    zip string,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Tables/Gold_layer/Customer_SCD1'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 82, Finished, Available, Finished)

DataFrame[]

In [13]:
df_src1=df_source.withColumn("hash_key",crc32(concat(*df_source.columns)))
display(df_src1)

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 83, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3f9466f2-bedc-4b52-98ae-b3e952ee9fa0)

In [14]:
from delta.tables import DeltaTable
tgt_path = "Tables/Gold_layer/Customer_SCD1"  
dbtable = DeltaTable.forPath(spark, tgt_path)
dbtable.toDF().show()

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 84, Finished, Available, Finished)

+-----------+-----------+---------+----------------+----------------+-----+------+----------+----------+--------------------+----------+--------------------+
|customer_id| first_name|last_name|         address|            city|state|   zip|  hash_key|created_by|        created_date|updated_by|        updated_date|
+-----------+-----------+---------+----------------+----------------+-----+------+----------+----------+--------------------+----------+--------------------+
|          5|      David|   Wilson|  202 Birch Blvd|       Vancouver|   BC|V5K0A1|3284019540|databricks|2025-05-04 05:28:...|databricks|2025-05-04 05:28:...|
|         17|     Joseph|    Green|  1616 Maple Ave|          Regina|   SK|S4P0A1|3916126300|databricks|2025-05-04 05:28:...|databricks|2025-05-04 05:28:...|
|         18|     Amelia|    Adams|     1717 Oak Dr|       Saskatoon|   SK|S7K0A1|2440028568|databricks|2025-05-04 05:28:...|databricks|2025-05-04 05:28:...|
|         19|Christopher|    Baker|    1818 Pine Rd|

In [15]:
df_src1=df_src1.alias("src").join(dbtable.toDF().alias("tgt"), ((col("src.customer_id") == col("tgt.customer_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_src1.show()

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 85, Finished, Available, Finished)

+-----------+----------+---------+--------------+----------+-----+----+--------+
|customer_id|first_name|last_name|       address|      city|state| zip|hash_key|
+-----------+----------+---------+--------------+----------+-----+----+--------+
|         87|   William| McDonald|8686 Maple Ave|Haileybury| NULL|NULL|    NULL|
|         87|   William| McDonald|8686 Maple Ave|Haileybury| NULL|NULL|    NULL|
|         87|   William| McDonald|8686 Maple Ave|Haileybury| NULL|NULL|    NULL|
|         87|   William| McDonald|8686 Maple Ave|Haileybury| NULL|NULL|    NULL|
|         87|   William| McDonald|8686 Maple Ave|Haileybury| NULL|NULL|    NULL|
+-----------+----------+---------+--------------+----------+-----+----+--------+



In [ ]:
df_src1_dedup = df_src1.dropDuplicates(["customer_id"])

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 86, Finished, Available, Finished)

In [16]:
dbtable.alias("tgt").merge(df_src1_dedup.alias("src"),"tgt.customer_id = src.customer_id")\
    .whenMatchedUpdate(set={"tgt.customer_id":"src.customer_id","tgt.first_name":"src.first_name","tgt.last_name":"src.last_name","tgt.address":"src.address","tgt.city":"src.city","tgt.state":"src.state","tgt.zip":"src.zip","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.customer_id":"src.customer_id","tgt.first_name":"src.first_name","tgt.last_name":"src.last_name","tgt.address":"src.address","tgt.city":"src.city","tgt.state":"src.state","tgt.zip":"src.zip","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
    

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 87, Finished, Available, Finished)

In [17]:
display(spark.read.format("delta").option("header","true").load(tgt_path))

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 88, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, def83dc1-de4c-40f9-8aa1-b804e02b0de9)

In [41]:
create_table_query = """
CREATE TABLE IF NOT EXISTS Loan_Payment (
  payment_id int,
  loan_id int,
  payment_date timestamp,
  payment_amount float,
  hash_key bigint,
  created_by string,
  created_date timestamp,
  updated_by string,
  updated_date timestamp
)
USING DELTA
LOCATION 'Tables/Gold_layer/Loan_Payment'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 89, Finished, Available, Finished)

DataFrame[]

In [19]:
from pyspark.sql.functions import *
df_source = spark.read.format("delta").load("Tables/dbo/loan_payments")
df_source.createOrReplaceTempView("source_view")
src_path="Tables/dbo/loan_payments"

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 90, Finished, Available, Finished)

In [20]:
df_src1=df_source.withColumn("hash_key",crc32(concat(*df_source.columns)))
display(df_src1)

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 91, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 952507cf-8f4e-49b7-9d96-424ae6e1035b)

In [21]:
from delta.tables import DeltaTable
tgt_path = "Tables/Gold_layer/Loan_Payment"  
dbtable = DeltaTable.forPath(spark, tgt_path)
dbtable.toDF().show()

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 92, Finished, Available, Finished)

+----------+-------+------------+--------------+--------+----------+------------+----------+------------+
|payment_id|loan_id|payment_date|payment_amount|hash_key|created_by|created_date|updated_by|updated_date|
+----------+-------+------------+--------------+--------+----------+------------+----------+------------+
+----------+-------+------------+--------------+--------+----------+------------+----------+------------+



In [22]:
df_src1=df_src1.alias("src").join(dbtable.toDF().alias("tgt"), ((col("src.payment_id") == col("tgt.payment_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_src1.show()

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 93, Finished, Available, Finished)

+----------+-------+------------+--------------+----------+
|payment_id|loan_id|payment_date|payment_amount|  hash_key|
+----------+-------+------------+--------------+----------+
|         5|     12|  2024-01-05|         300.0| 438074744|
|        10|     11|  2024-01-10|         550.0|3325951132|
|        31|     42|  2024-01-31|        1600.0|1567782063|
|        49|     40|  2024-02-18|        2500.0|2618666669|
|        56|     17|  2024-02-25|        2850.0|1586711593|
|        73|      4|  2024-03-13|        3700.0|2803722244|
|        75|     26|  2024-03-15|        3800.0|3353666919|
|        76|     37|  2024-03-16|        3850.0| 175174851|
|         7|     56|  2024-01-07|         400.0|1192820904|
|        20|     21|  2024-01-20|        1050.0|1615996424|
|        37|      8|  2024-02-06|        1900.0|1399685596|
|        40|     41|  2024-02-09|        2050.0|1343458395|
|        57|     28|  2024-02-26|        2900.0|2653151705|
|        60|     61|  2024-02-29|       

In [23]:
dbtable.alias("tgt").merge(df_src1.alias("src"),"tgt.payment_id = src.payment_id")\
    .whenMatchedUpdate(set={"tgt.payment_id":"src.payment_id","tgt.loan_id":"src.loan_id","tgt.payment_date":"src.payment_date","tgt.payment_amount":"src.payment_amount","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.payment_id":"src.payment_id","tgt.loan_id":"src.loan_id","tgt.payment_date":"src.payment_date","tgt.payment_amount":"src.payment_amount","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
     

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 94, Finished, Available, Finished)

In [24]:
display(spark.read.format("delta").option("header","true").load(tgt_path))

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 95, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 89e11295-60d3-481d-a991-f413c671ad53)

In [42]:
create_table_query = """
CREATE TABLE IF NOT EXISTS Loans1 (
  loan_id int,
  customer_id int,
  interest_rate float,
  loan_term int,
  loan_amount float,
  hash_key bigint,
  created_by string,
  created_date timestamp,
  updated_by string,
  updated_date timestamp
)
USING DELTA
LOCATION 'Tables/Gold_layer/Loans1'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 96, Finished, Available, Finished)

DataFrame[]

In [27]:
from pyspark.sql.functions import *
df_source = spark.read.format("delta").load("Tables/dbo/loans")
df_source.createOrReplaceTempView("source_view")
src_path="Tables/dbo/loans"

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 97, Finished, Available, Finished)

In [28]:
df_src1=df_source.withColumn("hash_key",crc32(concat(*df_source.columns)))
display(df_src1)

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 98, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6279522d-ec4c-450d-b494-32f96aa52ce4)

In [29]:
from delta.tables import DeltaTable
tgt_path = "Tables/Gold_layer/Loans1"  
dbtable = DeltaTable.forPath(spark, tgt_path)
dbtable.toDF().show()

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 99, Finished, Available, Finished)

+-------+-----------+-------------+---------+-----------+--------+----------+------------+----------+------------+
|loan_id|customer_id|interest_rate|loan_term|loan_amount|hash_key|created_by|created_date|updated_by|updated_date|
+-------+-----------+-------------+---------+-----------+--------+----------+------------+----------+------------+
+-------+-----------+-------------+---------+-----------+--------+----------+------------+----------+------------+



In [30]:
df_ln1=df_src1.alias("src").join(dbtable.toDF().alias("tgt"), ((col("src.loan_id") == col("tgt.loan_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_ln1.show()

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 100, Finished, Available, Finished)

+-------+-----------+-----------+-------------+---------+----------+
|loan_id|customer_id|loan_amount|interest_rate|loan_term|  hash_key|
+-------+-----------+-----------+-------------+---------+----------+
|     40|         19|    37500.0|          3.0|       24| 570153790|
|     74|         43|    30000.5|          4.5|       48|1215019206|
|     93|         79|   15000.25|          5.0|       36|2461134402|
|     13|         29|   15000.25|          5.0|       36|1193680727|
|     28|          7|    27500.0|          3.5|       24|1591891469|
|     69|         59|   32500.25|          5.0|       36|1838491308|
|     10|         92|    37500.5|          4.5|       48| 888718168|
|     39|         74|   32500.75|          6.5|       60|  16829702|
|     85|         65|   25000.25|          5.0|       36|1749183214|
|     86|         21|    17500.5|          4.0|       48|3548058147|
|     91|         77|   10000.75|          6.0|       60|3349540278|
|     99|         80|   32500.75| 

In [31]:
dbtable.alias("tgt").merge(df_ln1.alias("src"),"tgt.loan_id = src.loan_id")\
    .whenMatchedUpdate(set={"tgt.loan_id":"src.loan_id","tgt.customer_id":"src.customer_id","tgt.loan_amount":"src.loan_amount","tgt.interest_rate":"src.interest_rate","tgt.loan_term":"src.loan_term","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.loan_id":"src.loan_id","tgt.customer_id":"src.customer_id","tgt.loan_amount":"src.loan_amount","tgt.interest_rate":"src.interest_rate","tgt.loan_term":"src.loan_term","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
  

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 101, Finished, Available, Finished)

In [32]:
display(spark.read.format("delta").option("header","true").load(tgt_path))

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 102, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8bc5a5c9-235a-4352-b100-59e357b05e5e)

In [43]:
create_table_query = """
CREATE TABLE IF NOT EXISTS Transactions1 (
  transaction_id int,
  account_id int,
  transaction_date timestamp,
  transaction_amount float,
  transaction_type string,
  hash_key bigint,
  created_by string,
  created_date timestamp,
  updated_by string,
  updated_date timestamp
)
USING DELTA
LOCATION 'Tables/Gold_layer/Transactions1'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 103, Finished, Available, Finished)

DataFrame[]

In [34]:
from pyspark.sql.functions import *
df_source = spark.read.format("delta").load("Tables/dbo/transactions")
df_source.createOrReplaceTempView("source_view")
src_path="Tables/dbo/transactions"

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 104, Finished, Available, Finished)

In [35]:
df_src1=df_source.withColumn("hash_key",crc32(concat(*df_source.columns)))
display(df_src1)

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 105, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, df641330-49c6-4cb2-b145-396181bb5d5e)

In [36]:
from delta.tables import DeltaTable
tgt_path = "Tables/Gold_layer/Transactions1"  
dbtable = DeltaTable.forPath(spark, tgt_path)
dbtable.toDF().show()

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 106, Finished, Available, Finished)

+--------------+----------+----------------+------------------+----------------+--------+----------+------------+----------+------------+
|transaction_id|account_id|transaction_date|transaction_amount|transaction_type|hash_key|created_by|created_date|updated_by|updated_date|
+--------------+----------+----------------+------------------+----------------+--------+----------+------------+----------+------------+
+--------------+----------+----------------+------------------+----------------+--------+----------+------------+----------+------------+



In [37]:
df_ts1=df_src1.alias("src").join(dbtable.toDF().alias("tgt"), ((col("src.transaction_id") == col("tgt.transaction_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_ts1.show()

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 107, Finished, Available, Finished)

+--------------+----------+----------------+------------------+----------------+----------+
|transaction_id|account_id|transaction_date|transaction_amount|transaction_type|  hash_key|
+--------------+----------+----------------+------------------+----------------+----------+
|            21|        53|      2024-01-21|             100.5|         Deposit|1388575013|
|            44|        13|      2024-02-13|            300.25|      Withdrawal|2113995813|
|            79|        55|      2024-03-19|             325.0|         Deposit|3856079114|
|            96|        48|      2024-04-05|             175.0|      Withdrawal|2616276368|
|            31|        71|      2024-01-31|             100.5|         Deposit| 477929762|
|            35|        62|      2024-02-04|             250.0|         Deposit| 380560687|
|            52|        10|      2024-02-21|            200.75|      Withdrawal|1759883603|
|            62|        35|      2024-03-02|            200.75|      Withdrawal|

In [38]:
dbtable.alias("tgt").merge(df_ts1.alias("src"),"tgt.transaction_id = src.transaction_id")\
    .whenMatchedUpdate(set={"tgt.transaction_id":"src.transaction_id","tgt.account_id":"src.account_id","tgt.transaction_date":"src.transaction_date","tgt.transaction_amount":"src.transaction_amount","tgt.transaction_type":"src.transaction_type","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.transaction_id":"src.transaction_id","tgt.account_id":"src.account_id","tgt.transaction_date":"src.transaction_date","tgt.transaction_amount":"src.transaction_amount","tgt.transaction_type":"src.transaction_type","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
     

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 108, Finished, Available, Finished)

In [39]:
display(spark.read.format("delta").option("header","true").load(tgt_path))

StatementMeta(, d431f2e7-4fc2-443d-879e-8912e1bdbb00, 109, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6144383c-935d-4595-9f1a-d9e514bd0bad)